### Init (You'll need a .env file)

In [9]:
inputFileRoot = "PoC-v3-Data"
actuallySaveData = True

import os
import sys
import re
from typing import List, Dict, Any
from langchain_core.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from pydantic.v1 import BaseModel, Field
from openai import OpenAIError
from dotenv import load_dotenv
from tabulate import tabulate
from IPython.display import HTML, display
from openpyxl import Workbook, reader, load_workbook
import pandas as pd

# The API key belongs in the .env file
loaded = load_dotenv()

from langchain.globals import set_verbose, set_debug
set_debug(True)
set_verbose(True)

### Find the most recent data file

In [10]:
# Example usage:
#filename = "base.123.xlsx"
#new_filename = increment_version(filename)
#print(new_filename)  # Output: base.124.xlsx
def increment_version(filename):
    try:
        root, version, extension = filename.split('.')
        version = int(version) + 1
        return f"{root}.{version}.{extension}"
    except ValueError:
        raise ValueError("Filename format should be 'root.version.extension' and version should be an integer")

# Example usage:
#directory = "."
#root_segment = "PoC-v2-Data"
#highest_version_file = get_highest_version_file(directory, root_segment)
#print(highest_version_file)
def get_highest_version_file(directory, root_segment):
    highest_version = -1
    highest_version_file = None
    
    pattern = re.compile(rf'^{root_segment}\.(\d+)\.xlsx$')
    
    for filename in os.listdir(directory):
        match = pattern.match(filename)
        if match:
            version = int(match.group(1))
            if version > highest_version:
                highest_version = version
                highest_version_file = filename
    
    return highest_version_file

latestDataFileName = get_highest_version_file(".", inputFileRoot)

### Load Business Requirements from the spreadsheet

In [11]:
def get_BReqs_from_ws() -> pd.DataFrame:
    wb = reader.excel.load_workbook(filename=latestDataFileName, read_only=True, keep_vba=False, data_only=False, keep_links=True, rich_text=False)
    ws_br = wb["BReqs"]
    df = pd.DataFrame(ws_br.values)
    wb.close()
    df = df.rename(columns={0: "Name", 1: "Description"})
    #print(df.head())
    return df

def get_BReq_context():
    df = get_BReqs_from_ws()
    result = ""
    for index, row in df.iterrows():
        result += "Requirement-" + str(row["Name"]) + " is that " + str(row["Description"]) + ". "
    return result

### Generate some stories

In [12]:
class Story(BaseModel):
    title: str = Field(description="The title of the story")
    story_text: str = Field(description="The user story in the 'As a... I want to... So That...' format.")
    relevance_score: float = Field(description="The relevance of the story to the scenario. 0 is the worst, 10 is the best.")

class Stories(BaseModel):
    contents: List[Story] = Field(description="A list of Stories")

def create_chat_model(temperature: float = 0.0) -> ChatOpenAI:
    try:
        return ChatOpenAI(temperature=temperature)
    except OpenAIError as e:
        print(f"Error creating ChatOpenAI model: {e}")
        sys.exit(1)

def create_parser() -> PydanticOutputParser:
    return PydanticOutputParser(pydantic_object=Stories)

def create_chat_prompt(template: str) -> ChatPromptTemplate:
    system_message_prompt = SystemMessagePromptTemplate.from_template(template)
    return ChatPromptTemplate.from_messages([system_message_prompt])

def generate_stories(model: ChatOpenAI, chat_prompt: ChatPromptTemplate, parser: PydanticOutputParser,
                   input_data: Dict[str, Any]) -> Stories:
    try:
        print("***Chat Prompt***")
        print(chat_prompt)
        print("***Input Data***")
        print(input_data)
        prompt_and_model = chat_prompt | model
        result = prompt_and_model.invoke(input_data)
        return parser.parse(result.content)
    except OpenAIError as e:
        print(f"Error generating names: {e}")
        return Stories(contents=[])
    except ValueError as e:
        print(f"Error parsing output: {e}")
        print(f"The actual response was {result}")
        return Stories(contents=[])

def get_stories(feature: str):
    if 'OPENAI_API_KEY' not in os.environ:
        print("Error: OPENAI_API_KEY environment variable is not set.")
        sys.exit(1)

    breqs = get_BReq_context()
    print(breqs)

    template = """Generate the 10 most relevent agile user stories for a new custom software capability that offers a {feature} feature.
    You must follow the following principles:
    - The title should be between 3 and 8 words long.
    - The story should evoke the core goals of users and be between 15 and 50 words long.    
    - The story must be in the 'As a… I want to… So That…' format. 
    - Only stories that do not conflict with the following hard business requirements may be generated:
    {breqs}
    {format_instructions}
    """

    result = Stories(contents=[])
    try:
        model = create_chat_model()
        parser = create_parser()
        chat_prompt = create_chat_prompt(template)

        input_data = {
            "breqs": breqs,
            "feature": feature,
            "format_instructions": parser.get_format_instructions(),
        }
        #print(input_data)

        result = generate_stories(model, chat_prompt, parser, input_data)
        return result.contents
        
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        sys.exit(1)

### Run the procs so far

In [13]:
stories = get_stories("An online shopping cart")
#print(stories)

Requirement-Order-Timestamp is that All placed orders must be timestamped as 'order received' at a time between when the user clicked the 'Place order' button and when the payment processor authorized the payment.. Requirement-User-Authentication is that The software must require user authentication before checkout.. Requirement-Product-Selection is that Users may select multiple products to add to the shopping cart.. Requirement-Cart-Management is that Users must be able to view, add, update, or remove items in the shopping cart.. Requirement-Order-Summary is that The order summary shall display all items, quantities, and total price before payment.. Requirement-Payment-Gateway is that The software must integrate with at least one secure payment gateway.. Requirement-Order-Confirmation is that An order confirmation shall be displayed and emailed to the user upon successful payment.. Requirement-Cart-Security is that Users must not be permitted to save their cart for later.. Requiremen

In [14]:
stories_dicts = [story.__dict__ for story in stories]
table = tabulate(stories_dicts, tablefmt='html', headers="keys")
display(HTML(table))

title,story_text,relevance_score
View Cart Items,"As a shopper, I want to view all items in my shopping cart, so that I can review my selections before checkout.",9
Apply Discount Code,"As a customer, I want to apply a valid discount code to my order, so that I can receive a discounted price at checkout.",8
Remove Item from Cart,"As a user, I want to easily remove an item from my shopping cart, so that I can adjust my order before payment.",7
Receive Order Confirmation,"As a buyer, I want to receive an order confirmation after successful payment, so that I have a record of my purchase.",9
Calculate Taxes,"As a shopper, I want the software to calculate and display applicable taxes based on my location, so that I know the total cost of my order.",8
View Order Summary,"As a user, I want to see a summary of all items, quantities, and total price before payment, so that I can confirm my order details.",9
Receive Order Status Notifications,"As a customer, I want to receive notifications about order status changes via email or SMS, so that I am informed about my purchase.",8
Secure Checkout Process,"As a user, I want the software to require authentication before checkout, so that my payment information is secure.",9
Integrate with Payment Gateway,"As a shopper, I want the software to integrate with a secure payment gateway, so that I can make safe and reliable transactions.",9
Update Quantity in Cart,"As a buyer, I want to easily update the quantity of items in my shopping cart, so that I can adjust my order before payment.",7


### Save the new stories to the 'Stories' worksheet

In [15]:
def save_stories_to_excel(src_file_path: str, dst_file_path: str, sheet_name: str, stories: Stories):
    # Open the workbook and select the worksheet
    workbook = load_workbook(src_file_path)
    if sheet_name in workbook.sheetnames:
        sheet = workbook[sheet_name]
    else:
        sheet = workbook.create_sheet(sheet_name)

    # Delete all rows below the header
    sheet.delete_rows(2, sheet.max_row)

    # Define the headers
    #headers = ["Title", "Story Text", "Relevance Score"]
    #sheet.append(headers)
    
    # Write the data to the worksheet
    for story in stories:
        sheet.append([story.title, story.story_text, story.relevance_score])
    
    # Save the workbook
    workbook.save(dst_file_path)
    workbook.close()

# Save these stories to an Excel file
nextFileName = increment_version(latestDataFileName)
if actuallySaveData:
    save_stories_to_excel(latestDataFileName, nextFileName, "Stories", stories)

In [16]:
xls = pd.ExcelFile(nextFileName)
for sheet_name in xls.sheet_names:
    df = pd.read_excel(nextFileName, sheet_name=sheet_name)
    print(f"Sheet name: {sheet_name}")
    display(df)
xls.close()

Sheet name: BReqs


,Order-Timestamp,All placed orders must be timestamped as 'order received' at a time between when the user clicked the 'Place order' button and when the payment processor authorized the payment.
0,User-Authentication,The software must require user authentication ...
1,Product-Selection,Users may select multiple products to add to t...
2,Cart-Management,"Users must be able to view, add, update, or re..."
3,Order-Summary,"The order summary shall display all items, qua..."
4,Payment-Gateway,The software must integrate with at least one ...
5,Order-Confirmation,An order confirmation shall be displayed and e...
6,Cart-Security,Users must not be permitted to save their cart...
7,Discount-Handling,The system must apply valid discount codes to ...
8,Tax-Calculation,The software must calculate and display applic...
9,Customer-Notifications,Customers shall receive notifications about or...


Sheet name: Features


""


Sheet name: Stories


,Title,Story Text,Relevance Score
0,View Cart Items,"As a shopper, I want to view all items in my s...",9
1,Apply Discount Code,"As a customer, I want to apply a valid discoun...",8
2,Remove Item from Cart,"As a user, I want to easily remove an item fro...",7
3,Receive Order Confirmation,"As a buyer, I want to receive an order confirm...",9
4,Calculate Taxes,"As a shopper, I want the software to calculate...",8
5,View Order Summary,"As a user, I want to see a summary of all item...",9
6,Receive Order Status Notifications,"As a customer, I want to receive notifications...",8
7,Secure Checkout Process,"As a user, I want the software to require auth...",9
8,Integrate with Payment Gateway,"As a shopper, I want the software to integrate...",9
9,Update Quantity in Cart,"As a buyer, I want to easily update the quanti...",7
